In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE=224
BATCH_SIZE=32

In [4]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
   '/content/drive/MyDrive/data set/brain_tumor_project/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator=train_datagen.flow_from_directory(
   '/content/drive/MyDrive/data set/brain_tumor_project/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory(
    '/content/drive/MyDrive/data set/brain_tumor_project/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [5]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [6]:
#compile the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 370s 5s/step - loss: 0.5499 - accuracy: 0.7833 - val_loss: 0.2937 - val_accuracy: 0.8867
Epoch 2/5
75/75 [==============================] - 313s 4s/step - loss: 0.2289 - accuracy: 0.9104 - val_loss: 0.1441 - val_accuracy: 0.9433
Epoch 3/5
75/75 [==============================] - 316s 4s/step - loss: 0.1150 - accuracy: 0.9621 - val_loss: 0.1283 - val_accuracy: 0.9467
Epoch 4/5
75/75 [==============================] - 322s 4s/step - loss: 0.0549 - accuracy: 0.9842 - val_loss: 0.0232 - val_accuracy: 0.9983
Epoch 5/5
75/75 [==============================] - 314s 4s/step - loss: 0.0316 - accuracy: 0.9929 - val_loss: 0.0109 - val_accuracy: 0.9983


In [7]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model=load_model('/content/Model.h5')
test_image_path=('/content/drive/MyDrive/data set/brain_tumor_project/Brain_Tumor_Detection/test/pred/pred1.jpg')
img=image.load_img(test_image_path,target_size=(244,244,3))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)
img_array/=255.
prediction=model.predict(img_array)
if prediction < 0.5:
  print("Prediction: No Tumour (Probability:",prediction[0][0],")")
else:
  print("Prediction: Tumour present (Probability:",prediction[0][0],")")